# Setting Environment

In [ ]:
# downloading packages

# !pip3 install numpy
# !pip3 install pandas
# !pip3 install csv
# !pip3 install openpyxl

# Main Program

## Loading Package

In [45]:
import pandas as pd
import numpy as np
import os
import re

## Load the Data

In [46]:
# run these functions

def load_sheets(file_path, exclude_sheets):
    all_sheets = pd.ExcelFile(file_path).sheet_names
    importing_sheets = [sheet for sheet in all_sheets if sheet not in exclude_sheets]
    return pd.read_excel(file_path, sheet_name=importing_sheets, header=None)

def load_excel(directory):
    excel_files = {}
    for filename in os.listdir(directory):
        name = os.path.splitext(filename)[0]
        excel_files[name] = pd.read_excel(os.path.join(directory, filename), header=None)
    return excel_files

In [50]:
# loading the data

north_path = r"datasets\newest_พจนานุกรมภาษาถิ่นเหนือ.xlsx"
# south_path = r" "
# isan_path = r" "
# orst_path = r" "
# new_orst_path = r" "

north_df = pd.read_excel(north_path, sheet_name=None)
# south_df = pd.read_excel(south_path, sheet_name=None, header=None)
# isan_df = pd.read_excel(isan_path, sheet_name=None, header=None)
# orst_df = pd.read_excel(new_orst_path, sheet_name=None)
# orst_df = load_excel(orst_path)

In [51]:
# basic loading

new_df = pd.read_excel(r"datasets\north_ฉ.xlsx", header=None)

## Inspect the Data

In [57]:
# run to copy

north = north_df.copy()
# south = south_df.copy()
# isan = isan_df.copy()
# orst = orst_df.copy()

In [ ]:
df = new_df.copy()

In [58]:
north

{'ก':            คำ     POS                                           ความหมาย
 0          กก      น.                                โคน อย่าง โคนต้นไม้
 1          กก      น.                                   ชื่อเรียกนกเงือก
 2          กก      น.                       ชื่อแม่น้ำใน จังหวัดเชียงราย
 3          กก      น.        ชื่อหญ้าชนิดหนึ่งใช้ทำเลื่อ เรียกว่าเลื่อกก
 4          กก      ก.                       สับ บั่น ฟัน ตัด เช่น กก เกง
 ...       ...     ...                                                ...
 1075    ไกหัด      น.  ต้นไก ชื่อพรรณไม้ดอกคล้ายสารภีป่า ผลแดงเปลือกม...
 1076   ไก่ไห้      น.                                     ต้นตาลปัตรฤๅษี
 1077  ไก่อุ้ม      น.          ลูกไก่กำพร้าที่คนนำมาเลี้ยงไว้ ไก่คู่ขวัญ
 1078  ไก่อุ้ม  สำนวน.  มักนำมาใช้แซวคนหนุ่มสาวเวลาไม่สบายบ่อยว่าอยากไ...
 1079    ไก่อู      น.  ไก่ตัวใหญ่ หงอนใหญ่ ตัวผู้มีนหลายสี สีขนไม่สดใ...
 
 [1080 rows x 3 columns],
 'ข':               คำ POS                                           ความหมาย
 

In [53]:
# inspect the data
def inspect(num=10, *dfs):
  for df in dfs:
    for key in df.keys():
      print(f"{key}\n{df[key].head(num)}\n")

inspect(10, north)
# inspect(10, north, south, isan, orst)

ก
      คำ     POS                                           ความหมาย
0     กก      น.                                โคน อย่าง โคนต้นไม้
1     กก      น.                                   ชื่อเรียกนกเงือก
2     กก      น.                       ชื่อแม่น้ำใน จังหวัดเชียงราย
3     กก      น.        ชื่อหญ้าชนิดหนึ่งใช้ทำเลื่อ เรียกว่าเลื่อกก
4     กก      ก.                       สับ บั่น ฟัน ตัด เช่น กก เกง
5     กก  สำนวน.                           กกหางปล่อยวัด กกหัวกกหาง
6   กกดก      ก.                                 กระดกขึ้น เผยอขึ้น
7  กกหงก      ว.                                              ทนโท่
8     กง      น.             คันธนู คันกระสุน คันดีดฝ้าย หนังสะติ๊ก
9     กง      น.  หญ้าชนิดหนึ่งนิยมใช้ก้านดอกทำไม้กวาด เรียก หญ้...

ข
              คำ POS                                           ความหมาย
0         ขกอยาก  ก.                                         กระโดดขึ้น
1             ขง  น.                               กรุง เมืองอาณาบริเวณ
2             ขง  น.        กรง

## Cleaning

In [ ]:
# cleaning

"""make dataframe two columns"""
def cleaning(*dfs):
    for df in dfs:
        removed = ['ช่อง 1', 'ลูกคำ/ ความหมาย', 'แม่คำ', 'ความหมายลูกคำ คำสื่อ', 'headword', '2', '3']

        for key in df.keys():
            if df == orst:
                # remove the last column, and move the value in the second column to the first column
                df[key] = df[key].iloc[:, :-1]
                df[key].iloc[:, 0] = df[key].apply(lambda row: row.iloc[1] if pd.isnull(row.iloc[0]) else row.iloc[0], axis=1)
                df[key] = df[key].drop(columns=[df[key].columns[1]])
                df[key] = df[key].reset_index(drop=True)
                df[key].columns = ['word', 'meaning']
                df[key] = df[key].iloc[1:]
            
            else:
                # remove the third to fifth column
                df[key] = df[key][~df[key].apply(lambda row: any(string in str(val) for string in removed for val in row), axis=1)]
                df[key] = df[key].dropna(how='all')
                df[key] = df[key].drop(df[key].columns[2:], axis=1)
                df[key] = df[key].fillna("")
                df[key].columns = ['word', 'meaning']

cleaning(north)
# cleaning(north, south, isan, orst)

In [ ]:
# basic cleaning

def basic_cleaning(df):
    df = df.dropna(how='all')
    df = df.drop(df.columns[2:], axis=1)
    df = df.fillna("")
    df.columns = ['word', 'meaning']
    df = df.replace("", np.nan).ffill()
    return df

df = basic_cleaning(df)

In [ ]:
df

## Checking

In [55]:
# checking
comma_value = []
missing_value = []
POS_value = []

def checking(df):
    regex = r',(?!(?:[^(]*\)))'
    POS = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|บุ\.|\*ส\.|สำ\.|\*สำ\.|ล\.|สำนวน\.|สำนวน\.\*|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|สรรพ\.|อ\.|อุทาน\.|\*อ\.|อุ\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|\?)(.*)$"

    for key in df.keys():
        # checking comma in "word" column
        for value in df[key]['word']:
            if re.search(regex, value):
                comma_value.append(value)
         
        for value in df[key]['meaning']:
            matches = re.findall(POS, value)
            # check if a word has more than one POS
            if len(matches) > 1:
                POS_value.append(value)
            
            # check if a meaning or definition isn't well-written
            elif len(matches) < 1 and value != "" and value.isspace() == False:
                missing_value.append(value)

    print(comma_value if len(comma_value) > 0 else None)
    print(POS_value if len(comma_value) > 0 else None)

checking(north)
# checking(south)
# checking(isan)
# checking(orst)

None
None


In [ ]:
# basic checking

comma_value = []
missing_value = []
POS_value = []

def basic_checking(df):
    regex = r',(?!(?:[^(]*\)))'
    POS = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|บุ\.|\*ส\.|สำ\.|\*สำ\.|ล\.|สำนวน\.|สำนวน\.\*|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|สรรพ\.|อ\.|อุทาน\.|\*อ\.|อุ\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|\?)(.*)$"

    for value in df['word']:
        if re.search(regex, value):
            comma_value.append(value)
        
    for value in df['meaning']:
        matches = re.findall(POS, value)
        if len(matches) > 1:
            POS_value.append(value)
        elif len(matches) < 1 and value != "" and value.isspace() == False:
            missing_value.append(value)

    print(comma_value if len(comma_value) > 0 else None)
    print(POS_value if len(comma_value) > 0 else None)

basic_checking(df)

##  Orst Splitting

In [ ]:
# another splitting

def orst_split(df):
    for key in df.keys():
        # split comma in the first column
        df[key]['word'] = df[key]['word'].str.split(',')
        df[key] = df[key].explode('word')
        df[key]['word'] = df[key]['word'].str.strip()

        # split semi-colon in the third column
        df[key]['meaning'] = df[key]['meaning'].str.split(';')
        df[key] = df[key].explode('meaning')
        df[key]['meaning'] = df[key]['meaning'].str.strip()

        # # filling and reseting index
        df[key] = df[key].ffill()
        df[key].drop_duplicates(inplace=True)
        df[key] = df[key].reset_index(drop=True)

# orst_split(orst)

## Split POS

In [ ]:
# splitting POS

def rearrange(df):
  for key in df.keys():
    pattern = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|\*ส\.|สำ\.|\*สำ\.|สำนวน\.|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|ล\.|สรรพ\.|อ\.|\*อ\.|อุ\.|อุทาน\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|บุ\.|\?)(.*)$"
    
    # create new columns
    df[key]["คำ"] = df[key]['word']
    df[key]["POS"] = ""
    df[key]["ความหมาย"] = ""

    # ensure that a meaning begins with POS or a pattern
    if df[key]['meaning'].str.startswith((
      "น.", "*น.", "นิ.", "ก.", "*ก.", "ว.", "*ว.", "สัน.", "*สำ.", "สำ.", "ส.", "*ส.", "ความเปรียบ.", "คำถาม.", "คำยกย่อง.", "อุทาน.",
      "?", "บุพ.", "บ.", "บุ.", "บุรพ.", "สำนวน.", "สำนวน.*", "ลัก.", "สรรพ.", "อ.", "*อ.", "อุ.", "อุทาน.", "คำลงท้าย.")).any():
      
      # extract POS to get two new columns
      df[key][["POS", "ความหมาย"]] = df[key]['meaning'].str.extract(pattern, expand=True)
      df[key]["ความหมาย"] = df[key].apply(lambda row: row[2] if row["POS"] == "" else row["ความหมาย"], axis=1)

    pos_mask = df[key]['meaning'].str.match(pattern, na=False)
    df[key]["POS"] = df[key]["POS"].fillna("")
    df[key].loc[~pos_mask, "ความหมาย"] = df[key].loc[~pos_mask, 'meaning']
    df[key] = df[key].iloc[:, 2:]

# rearrange(north)
# rearrange(south)
# rearrange(isan)
# rearrange(orst)

def clean_whitespace(df):
  for key in df.keys():
    for column in df[key].columns:
      if df[key][column].dtype == 'object':
        df[key][column] = df[key][column].apply(
          lambda text: ' '.join(text.split()) if isinstance(text, str) else text
        )
  return df

# north = clean_whitespace(north)
# south = clean_whitespace(south)
# isan = clean_whitespace(isan)
# orst = clean_whitespace(orst)

def no_whitespace(df):
  for key in df.keys():
      df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
                lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
            )

# no_whitespace(north)
# no_whitespace(south)
# no_whitespace(isan)
# no_whitespace(orst)

In [ ]:
# refactored splitting POS

def rearrange(df):
    # Part 1: Splitting POS
    for key in df.keys():
        pattern = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|\*ส\.|สำ\.|\*สำ\.|สำนวน\.|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|ล\.|สรรพ\.|อ\.|\*อ\.|อุ\.|อุทาน\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|บุ\.|\?)(.*)$"
        
        df[key]["คำ"] = df[key]['word']
        df[key]["POS"] = ""
        df[key]["ความหมาย"] = ""

        if df[key]['meaning'].str.startswith((
            "น.", "*น.", "นิ.", "ก.", "*ก.", "ว.", "*ว.", "สัน.", "*สำ.", "สำ.", "ส.", "*ส.", 
            "ความเปรียบ.", "คำถาม.", "คำยกย่อง.", "อุทาน.", "?", "บุพ.", "บ.", "บุ.", "บุรพ.", 
            "สำนวน.", "สำนวน.*", "ลัก.", "สรรพ.", "อ.", "*อ.", "อุ.", "อุทาน.", "คำลงท้าย.")).any():
        
            df[key][["POS", "ความหมาย"]] = df[key]['meaning'].str.extract(pattern, expand=True)
            df[key]["ความหมาย"] = df[key].apply(lambda row: row[2] if row["POS"] == "" else row["ความหมาย"], axis=1)

        pos_mask = df[key]['meaning'].str.match(pattern, na=False)
        df[key]["POS"] = df[key]["POS"].fillna("")
        df[key].loc[~pos_mask, "ความหมาย"] = df[key].loc[~pos_mask, 'meaning']
        df[key] = df[key].iloc[:, 2:]

        # Part 2: Clean whitespace
        for column in df[key].columns:
            if df[key][column].dtype == 'object':
                df[key][column] = df[key][column].apply(
                    lambda text: ' '.join(text.split()) if isinstance(text, str) else text
                )

        # Part 3: Remove specific whitespace
        df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
            lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
        )
    
    return df

# north = rearrange(north)
# south = rearrange(south)
# isan = rearrange(isan)
# orst = rearrange(orst)

In [ ]:
# basic splitting POS

def basic_rearrange(df):
    pattern = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|\*ส\.|สำ\.|\*สำ\.|สำนวน\.|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|ล\.|สรรพ\.|อ\.|\*อ\.|อุ\.|อุทาน\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|บุ\.|\?)(.*)$"
    
    df = df.copy()
    
    df["คำ"] = df['word']
    df["POS"] = ""
    df["ความหมาย"] = ""

    if df['meaning'].str.startswith((
        "น.", "*น.", "นิ.", "ก.", "*ก.", "ว.", "*ว.", "สัน.", "*สำ.","สำ.", "ส.",
        "*ส.", "ความเปรียบ.", "คำถาม.", "คำยกย่อง.", "อุทาน.", "?", "บุพ.", "บ.",
        "บุ.", "บุรพ.", "สำนวน.", "สำนวน.*", "ลัก.", "สรรพ.", "อ.", "*อ.", "อุ.",
        "อุทาน.", "คำลงท้าย.")).any():
        
        df[["POS", "ความหมาย"]] = df['meaning'].str.extract(pattern, expand=True)
        df["ความหมาย"] = df.apply(lambda row: row[2] if row["POS"] == "" else row["ความหมาย"], axis=1)

    pos_mask = df['meaning'].str.match(pattern, na=False)
    df["POS"] = df["POS"].fillna("")

    df.loc[~pos_mask, "ความหมาย"] = df.loc[~pos_mask, 'meaning']
    df['ความหมาย'] = df['ความหมาย'].apply(
        lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
    )
    df = df.iloc[:, 2:]
    df = df.reset_index(drop=True)
    return df

df = basic_rearrange(df)

In [56]:
north

{'ก':          word meaning
 0          กก      น.
 1          กก      น.
 2          กก      น.
 3          กก      น.
 4          กก      ก.
 ...       ...     ...
 1075    ไกหัด      น.
 1076   ไก่ไห้      น.
 1077  ไก่อุ้ม      น.
 1078  ไก่อุ้ม  สำนวน.
 1079    ไก่อู      น.
 
 [1080 rows x 2 columns],
 'ข':             word meaning
 0         ขกอยาก      ก.
 1             ขง      น.
 2             ขง      น.
 3             ขง      ก.
 4             ขง      น.
 ...          ...     ...
 1280   ไข่เหลือง      น.
 1281   ไข่เหลือง      น.
 1282  ไข่เหาหลวง      น.
 1283     ไข่อ็อก      น.
 1284     ไข่ฮ่วน      น.
 
 [1285 rows x 2 columns],
 'ฃ':       word meaning
 0  ฃาบอยาบ      ก.,
 'ค':          word meaning
 0          ค็    สัน.
 1          คก      น.
 2        คกงก      ว.
 3        คกงก      ว.
 4       คกภิก      น.
 ..        ...     ...
 796   ไคร่หัว      ก.
 797  ไคร่อยาก      ก.
 798       ไคล      น.
 799       ไคล      ว.
 800    ไคลคลา      ก.
 
 [801 rows x 2 co

## Fill Blank Values

In [ ]:
# fill blank values
def fill_blank_words(iter=2, *dfs):
    for df in dfs:
        for _ in range(iter):
            for key in df.keys():
                df[key] = df[key].dropna(how='all')
                df[key] = df[key].infer_objects(copy=False)
                df[key] = df[key].replace(r'^\s*$', np.nan, regex=True) 
                df[key] = df[key].ffill()
                df[key].drop_duplicates(inplace=True)
                df[key] = df[key].reset_index(drop=True)
        return df

# fill_blank_words(2, north, south, isan, orst)

## Enumeration

In [ ]:
# enumeration

def counting(df):
    def clean_text(input_text):
        pattern = r"[๐๑๒๓๔๕๖๗๘๙]|(\(.*?\))"
        cleaned_text = re.sub(pattern, "", input_text).strip()
        return cleaned_text

    for key in df.keys():
        df[key]['คำ'] = df[key]['คำ'].apply(clean_text)
                
        df[key]['occurrence'] = df[key].groupby('คำ').cumcount()
        df[key]['คำ'] = df[key]['คำ'] + " " + df[key]['occurrence'].astype(str)
        df[key] = df[key].drop(columns=['occurrence'])

        df[key]['คำ'] = df[key]['คำ'].apply(lambda x: ' '.join(x.replace("\u200b", "").split()))
                
        df[key]['ความหมาย'] = df[key]['ความหมาย'].astype(str)
        df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", "")))

        df[key] = df[key].drop_duplicates()

## Quick Cleaning

In [ ]:
# quick importing

import pandas as pd

path = r" "

df = pd.read_excel(path, sheet_name=None)

In [ ]:
# quick cleaning

import re

def no_more_whitespace(df):
    for key in df.keys():
        df[key].drop_duplicates(inplace=True)
        df[key] = df[key].reset_index(drop=True)
        df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
                lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
            )

# no_more_whitespace(df)

## Output

In [59]:
# save as csv

def df_to_csv(dfs, region, path):
    for sheet, df in dfs.items():
        file = f"{path}/{region}_{sheet}.csv"
        df.to_csv(file, index=False, encoding='utf-8-sig')

regions = {
    'north': north,
    # 'cen': orst,
    # 'south': south,
    # 'isan': isan,
    }

for region, dfs in regions.items():
    df_to_csv(dfs, region, r"output")

In [ ]:
# save as xlsx file

def df_to_xlsx(dfs, region, path):
    with pd.ExcelWriter(f"{path}/{region}.xlsx") as writer:
        for sheet, df in dfs.items():
            df.to_excel(writer, sheet_name=sheet, index=False)

regions = {
    # 'cleaned_พจนานุกรมภาษาถิ่นเหนือ': north,
    # 'cleaned_พจนานุกรมภาษาถิ่นใต้': south,
    # 'cleaned_พจนานุกรมภาษาถิ่นอีสาน': isan,
    # 'cleaned_พจนานุกรมภาษาถิ่นกลาง': orst,
    }

for region, dfs in regions.items():
    df_to_xlsx(dfs, region, r" ")

In [ ]:
# basic saving

df.to_csv(r"output\north_ฉ.csv", index=False, encoding='utf-8-sig')

## Quick Enumeration

In [61]:
# importing

import pandas as pd
import os
import re
from pathlib import Path

In [62]:
# enumeration csv to csv

input_path = r"output"
output_path = r"output\north"
Path(output_path).mkdir(parents=True, exist_ok=True)

def clean_text(input_text):
    pattern = r"[๐๑๒๓๔๕๖๗๘๙]|(\(.*?\))"
    cleaned_text = re.sub(pattern, "", input_text).strip()
    return cleaned_text

for filename in os.listdir(input_path):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_path, filename)
        df = pd.read_csv(input_file)
        df['คำ'] = df['คำ'].apply(clean_text)
        
        # enumeration
        df['occurrence'] = df.groupby('คำ').cumcount()
        df['คำ'] = df['คำ'] + " " + df['occurrence'].astype(str)
        df = df.drop(columns=['occurrence'])

        # cleaning 
        df['คำ'] = df['คำ'].apply(lambda x: ' '.join(x.replace("\u200b", "").split()))
        
        df['ความหมาย'] = df['ความหมาย'].astype(str)
        df['ความหมาย'] = df['ความหมาย'].apply(lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", "")))

        df = df.drop_duplicates()

        output_file = os.path.join(output_path, filename)
        df.to_csv(output_file, index=False)